<a href="https://colab.research.google.com/github/alyssaimani/Drug_Discovery_Acetylcholinesterase/blob/main/Drug_Discovery_acetylcholinesterase_Regression_with_ChemBERTa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tuning Regression ChemBERTa on SMILES Data for Human Acetylcholinesterase

In [2]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [34]:
import pandas as pd
df = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/Bioinformatics/Acetylcholinesterase/acetylcholinesterase_bioactivity_data_3class_pIC50.csv")

## Split Dataset

In [35]:
columnNames = ["canonical_smiles","pIC50"]
smiles_reg = pd.DataFrame(data =df, columns=columnNames)
smiles_reg

,canonical_smiles,pIC50
0,CCOc1nn(-c2cccc(OCc3ccccc3)c2)c(=O)o1,6.124939
1,O=C(N1CCCCC1)n1nc(-c2ccc(Cl)cc2)nc1SCC1CC1,7.000000
2,CN(C(=O)n1nc(-c2ccc(Cl)cc2)nc1SCC(F)(F)F)c1ccccc1,4.301030
3,O=C(N1CCCCC1)n1nc(-c2ccc(Cl)cc2)nc1SCC(F)(F)F,6.522879
4,CSc1nc(-c2ccc(OC(F)(F)F)cc2)nn1C(=O)N(C)C,6.096910
...,...,...
3592,O=C(NCCCCCCNc1c2c(nc3ccccc13)CCCC2)c1cc2ccccc2o1,6.372634
3593,O=C(NCCCCCCNc1c2c(nc3ccccc13)CCC2)c1cc2ccccc2o1,6.263603
3594,CCN(CCCCCCCCc1cccc(OC)c1)Cc1ccccc1OC,5.247952
3595,CC(C)=CC[C@H]1C[C@@]2(CC=C(C)C)C(=O)O[C@@](CC=...,5.017729


In [36]:
from sklearn.model_selection import train_test_split

train_data_reg, test_data_reg, train_labels_reg, test_labels_reg = train_test_split(smiles_reg['canonical_smiles'], smiles_reg['pIC50'], test_size=0.2, random_state=42)
train_data_reg, val_data_reg, train_labels_reg, val_labels_reg= train_test_split(train_data_reg, train_labels_reg, test_size=0.2, random_state=42)

In [37]:
train_reg = pd.concat([train_data_reg, train_labels_reg], axis=1)
test_reg = pd.concat([test_data_reg, test_labels_reg], axis=1)
val_reg = pd.concat([val_data_reg, val_labels_reg], axis=1)

In [38]:
print("Train Dataset: {}".format(train_reg.shape))
print("Eval Dataset: {}".format(val_reg.shape))
print("TEST Dataset: {}".format(test_reg.shape))

Train Dataset: (2301, 2)
Eval Dataset: (576, 2)
TEST Dataset: (720, 2)


## Import Libraries

In [8]:
!pip install --pre deepchem
import deepchem
from rdkit import Chem

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 773.0/773.0 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.7/29.7 MB 42.6 MB/s eta 0:00:00


In [9]:
!pip install transformers
!pip install simpletransformers
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 42.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 22.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.5/250.5 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 79.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 66.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [10]:
from simpletransformers.classification import ClassificationModel
import logging
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

## Build Classifucation Model with ChemBERTa

In [11]:
# Check avaibility CUDA
import torch

print(torch.cuda.is_available())

True


In [39]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
# Enabling regression
# Setting optional model configuration
model_args = ClassificationArgs()
model_args.num_train_epochs = 1
model_args.regression = True
model_args.evaluate_each_epoch = True
model_args.evaluate_during_training_verbose = True
model_args.no_save = True
model_args.num_train_epochs = 10
model_args.auto_weights = True

model = ClassificationModel('roberta', 'seyonec/PubChem10M_SMILES_BPE_396_250', num_labels=1, args=model_args) 

Some weights of the model checkpoint at seyonec/PubChem10M_SMILES_BPE_396_250 were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.dense.bias', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at seyonec/PubChem10M_SMILES_BPE

In [40]:
print(model.tokenizer)

RobertaTokenizerFast(name_or_path='seyonec/PubChem10M_SMILES_BPE_396_250', vocab_size=7924, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'sep_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'pad_token': AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'cls_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=True)}, clean_up_tokenization_spaces=True)


In [41]:
import wandb
wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

## Train the Model

In [44]:
#Train the model
model.train_model(train_reg, eval_df=val_reg, output_dir='/content/PubChem_acetylcholinesterase_reg', args={'wandb_project': 'Drug Discovery - Acetylcholinesterase'})

/usr/local/lib/python3.9/dist-packages/simpletransformers/classification/classification_model.py:612: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/2301 [00:00<?, ?it/s]

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Training loss,█▃▆▄▂▂▃▃▂▄▂▂▂▂▃▃▂▁▃▂▂▁▁▂▁▁▂▁▂▁▁▁▂▁▂▁▁▁▁▁
global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇██
lr,▄▆███▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▁▁
Training loss,0.3128
global_step,2150
lr,0.0


Running Epoch 0 of 10:   0%|          | 0/288 [00:00<?, ?it/s]

...

Running Epoch 6 of 10:   0%|          | 0/288 [00:00<?, ?it/s]

Running Epoch 7 of 10:   0%|          | 0/288 [00:00<?, ?it/s]

Running Epoch 8 of 10:   0%|          | 0/288 [00:00<?, ?it/s]

Running Epoch 9 of 10:   0%|          | 0/288 [00:00<?, ?it/s]

(2880, 1.041851479451806)

In [49]:
import sklearn
# R Squared
result, model_outputs, wrong_predictions = model.eval_model(test_reg, r2=sklearn.metrics.r2_score, mse=sklearn.metrics.mean_squared_error)

/usr/local/lib/python3.9/dist-packages/simpletransformers/classification/classification_model.py:1454: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/720 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/90 [00:00<?, ?it/s]

In [50]:
result

{'r2': 0.6896530347995424,
 'mse': 0.7749628855919103,
 'eval_loss': 0.7749628886580467}

In [51]:
import pickle

# Save the model
with open('regression.pkl', 'wb') as f:
    pickle.dump(model, f)